In [ ]:
'''
Created on October 282022

@auther: Yan

'''

from tensorflow import keras 
tf.compat.v1.enable_eager_execution()
import tensorflow as tf
from tensorflow.keras import models
from keras.layers.core import Dense, Dropout, Activation
import pandas as pd
import numpy as np
import os

In [ ]:
# Load Data

df_train = pd.read_excel(r"./Data_FCFP.xlsx")
df_test =  pd.read_excel(r"./Data_FCFP.xlsx")

X_train = np.asarray(df_train.iloc[:193,2:], dtype='float64')
train_targets = np.asarray(df_train.iloc[:193,1], dtype='float64')
X_test = np.asarray(df_test.iloc[193:,2:], dtype='float64')
test_targets = np.asarray(df_test.iloc[193:,1], dtype='float64')



In [ ]:
# Use GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from keras.backend.tensorflow_backend import set_session
from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
K.set_session(tf.compat.v1.Session(config=config))

In [ ]:
'''
Build the model
'''

from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1], )))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

def compile_model(model):
    from keras.utils import multi_gpu_model
    model = multi_gpu_model(model,1)
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

for k in range(50):
    
    '''
    five-fold cross validation
    '''

    num_epochs = 500
    batch_size = 8
    y_pred_5cv = []
    y_exp_5cv = []

    from sklearn.model_selection import KFold
    from sklearn.metrics import r2_score

    kf = KFold(n_splits=5, shuffle=True)
    i = 0
    expvspred_5cv_index = []
    change = np.zeros(len(X_train))

    for train_index, test_index in kf.split(X_train):
        print('Train:', train_index, 'Test:', test_index)

        partial_train_data, val_data = X_train[train_index], X_train[test_index]
        partial_train_targets, val_targets = train_targets[train_index], train_targets[test_index]
        

        model = build_model()
        model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size = batch_size)
        change[test_index] = model.predict(val_data).reshape(-1,)
        
        y_exp_5cv = train_targets
        y_pred_5cv = change 

        model.save('deep_learning_cell_uptake_5cv{}.h5'.format(i))
        i += 1

    y_exp_5cv = np.asarray(y_exp_5cv).reshape((len(X_train), ))
    y_pred_5cv = np.asarray(y_pred_5cv).reshape((len(X_train), ))
    r2_5CV = r2_score(y_exp_5cv, y_pred_5cv)
    print("==="*5,"Range:",k,"R2_5CV:",r2_5CV,"==="*35)


    '''
    Test set validation
    '''

    model_test = build_model()
    history = model_test.fit(X_train, train_targets, epochs=num_epochs, batch_size = batch_size,validation_data=(X_test,test_targets))
    model_test.save('Deep_learning_cell_uptake_test.h5')
    y_test_pred = model_test.predict(X_test)
    y_test_pred = np.asarray(y_test_pred).reshape(len(X_test), )
    r2_test = r2_score(test_targets, y_test_pred)
    print("==="*5,"Range:",k,"R2_test:",r2_test,"==="*65)
    
    if 0.60<r2_5CV<0.85 and 0.60<r2_test<0.85:
        print("right!!")
        print("5cv:",r2_5CV,"test:",r2_test)
        os.mkdir("R_{}_s{}".format(k,r2_5CV))
        expvspred_5cv = {'Exp': y_exp_5cv, 'Pred': y_pred_5cv}
        pd.DataFrame(expvspred_5cv).to_excel('./R_{}_s{}/5CV_{}.xlsx'.format(k,r2_5CV,r2_5CV))
        expvspred_test = {'Exp': test_targets, 'Pred': y_test_pred}
        pd.DataFrame(expvspred_test).to_excel('./R_{}_s{}/Test_{}.xlsx'.format(k,r2_5CV,r2_test))
        
        history.dict = history.history
        mae_values = history.dict['loss']
        val_mae_values = history.dict['val_loss']
        pd.DataFrame(mae_values).to_excel('./R_{}_s{}/training_loss.xlsx'.format(k,r2_5CV))
        pd.DataFrame(val_mae_values).to_excel('./R_{}_s{}/val_loss.xlsx'.format(k,r2_5CV))

        import matplotlib.pyplot as plt

        plt.clf()

        plt.plot(range(1, len(mae_values) +1 ), mae_values, 'r', label='Training loss')
        plt.plot(range(1, len(mae_values) + 1), val_mae_values, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Mean absolute error')
        plt.legend()
        plt.savefig('./R_{}_s{}/Training and validation loss.png'.format(k,r2_5CV), dpi=600)
    k+=1 
